In [1]:
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import chisquare, chi2
sys.path.append("../src")


In [32]:
xi = 5.0
prefix = f"/nfs/dust/luxe/user/spatarod/ConnectionListStudy/e0gpc/{xi}/smeared"

BX = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

sub_qubo_size = 7

In [33]:
num_BX_impact_list_eigensolver_completed = 0
num_BX_connection_list_eigensolver_completed = 0

num_BX_impact_list_TwoLocal_completed = 0
num_BX_connection_list_TwoLocal_completed = 0

num_BX_impact_list_HamiltonianDriven_completed = 0
num_BX_connection_list_HamiltonianDriven_completed = 0

for folder in os.listdir(prefix):
    if "example" in folder:
        for subfolder in os.listdir(f"{prefix}/{folder}"):
            if f"eigensolver_{sub_qubo_size}q_impact_list_reverse" in subfolder:
                is_complete = False
                for file in os.listdir(f"{prefix}/{folder}/{subfolder}"):
                    if "reco_xplet_list_ambiguity_solved" in file:
                        is_complete = True
                        num_BX_impact_list_eigensolver_completed +=1
                        
            if f"eigensolver_{sub_qubo_size}q_connection_list" in subfolder:
                is_complete = False
                for file in os.listdir(f"{prefix}/{folder}/{subfolder}"):
                    if "reco_xplet_list_ambiguity_solved" in file:
                        is_complete = True
                        num_BX_connection_list_eigensolver_completed +=1
                        
            if f"VQE_IdealQasmSim_{sub_qubo_size}q_TwoLocal_linear_NFT_impact_list_reverse" in subfolder:
                is_complete = False
                for file in os.listdir(f"{prefix}/{folder}/{subfolder}"):
                    if "reco_xplet_list_ambiguity_solved" in file:
                        is_complete = True
                        num_BX_impact_list_TwoLocal_completed +=1   

            if f"VQE_IdealQasmSim_{sub_qubo_size}q_NFT_hamiltonian_driven_impact_list_reverse" in subfolder:
                is_complete = False
                for file in os.listdir(f"{prefix}/{folder}/{subfolder}"):
                    if "reco_xplet_list_ambiguity_solved" in file:
                        is_complete = True
                        num_BX_impact_list_HamiltonianDriven_completed +=1   

            if f"VQE_IdealQasmSim_{sub_qubo_size}q_TwoLocal_linear_NFT_connection_list" in subfolder:
                is_complete = False
                for file in os.listdir(f"{prefix}/{folder}/{subfolder}"):
                    if "reco_xplet_list_ambiguity_solved" in file:
                        is_complete = True
                        num_BX_connection_list_TwoLocal_completed +=1
                        
                        
            if f"VQE_IdealQasmSim_{sub_qubo_size}q_NFT_hamiltonian_driven_connection_list" in subfolder:
                is_complete = False
                for file in os.listdir(f"{prefix}/{folder}/{subfolder}"):
                    if "reco_xplet_list_ambiguity_solved" in file:
                        is_complete = True
                        num_BX_connection_list_HamiltonianDriven_completed +=1
                        
print(num_BX_impact_list_eigensolver_completed)
print(num_BX_connection_list_eigensolver_completed)
print(num_BX_impact_list_TwoLocal_completed)
print(num_BX_connection_list_TwoLocal_completed)
print(num_BX_impact_list_HamiltonianDriven_completed)
print(num_BX_connection_list_HamiltonianDriven_completed)

10
10
10
10
10
10


In [16]:
def get_eff_and_frate(reco_xplets, gen_xplets):

    matched_tracks = 0
    fake_tracks = 0

    matched_tracks_bookkeeping = set()
    
    fake_chi_squared = []
    matched_chi_squared = []
    
    for track in reco_xplets:
        matched = False
        p_id = None
        ids = set(track.particle_ids.values())
        for test_id in ids:
            count = 0
            for particle_id in track.particle_ids.values():
                if test_id == particle_id:
                    count += 1
            if count >= 3:
                p_id = test_id
                matched = True
        if matched:
            if p_id not in matched_tracks_bookkeeping:
                matched_tracks += 1
                matched_tracks_bookkeeping.add(p_id)
                matched_chi_squared.append(fit_lin_track(track))
            else:
                pass
        else:
            fake_tracks += 1
            fake_chi_squared.append(fit_lin_track(track))
   
    return matched_tracks / len(gen_xplets), fake_tracks / len(reco_xplets), matched_chi_squared, fake_chi_squared

In [ ]:
def lin_func(x, a, b):
    """Linear function with slope a and bias b
    :param x: data points
    :param a: slope
    :param b: bias
    """
    return a * x + b




def fit_lin_track(track):
    """Linear fit of the track in xz and yz direction. Chi squared values are averaged. Chi squared and p-value
    attributes are set for the xplet.
    """
    x = [value[0] for value in track.coordinates.values()]
    y = [value[1] for value in track.coordinates.values()]
    z = [value[2] for value in track.coordinates.values()]

    popt_xz, _ = curve_fit(lin_func, z, x)
    popt_yz, _ = curve_fit(lin_func, z, y)


    chi_xz, _ = chisquare(x, f_exp=[popt_xz[0] * z_i + popt_xz[1] for z_i in z], ddof=len(x) - 2)
    chi_yz, _ = chisquare(y, f_exp=[popt_yz[0] * z_i + popt_yz[1] for z_i in z], ddof=len(y) - 2)

    return 0.5 * (chi_xz + chi_yz), chi2.sf(0.5 * (chi_xz + chi_yz), df=len(x) - 1 - (len(x) - 2))
       

In [ ]:
xi_7_eff_connection_list = []
xi_7_connection_list_chi_squared_matched = []
xi_7_connection_list_chi_squared_fake = []

xi_7_eff_impact_list = []
xi_7_impact_list_chi_squared_matched = []
xi_7_impact_list_chi_squared_fake = []

xi_7_frate_connection_list = []
xi_7_frate_impact_list = []

for folder in os.listdir(prefix):
    if "example" in folder:
        for subfolder in os.listdir(f"{prefix}/{folder}"):
            if f"eigensolver_{sub_qubo_size}q_impact_list_reverse" in subfolder:
                reco_xplets = np.load(f"{prefix}/{folder}/{subfolder}/reco_xplet_list_ambiguity_solved.npy", allow_pickle=True)
                gen_xplets = np.load(f"{prefix}/" + "_".join(["_".join(folder.split("_")[0:3]),"sl", "gen_xplet_list.npy"]),
                                     allow_pickle=True)
                statistics = get_eff_and_frate(reco_xplets, gen_xplets)
                xi_7_eff_impact_list.append(statistics[0])
                xi_7_frate_impact_list.append(statistics[1])
                
                for chi in statistics[2]:
                    xi_7_eff_impact_list_chi_squared_matched.append(chi)
                for chi in statistics[3]:
                    xi_7_eff_impact_list_chi_squared_fake.append(chi)

           
            if f"eigensolver_{sub_qubo_size}q_connection_list" in subfolder:
                reco_xplets = np.load(f"{prefix}/{folder}/{subfolder}/reco_xplet_list_ambiguity_solved.npy",
                                    allow_pickle=True)
                gen_xplets = np.load(f"{prefix}/" + "_".join(["_".join(folder.split("_")[0:3]),"sl", "gen_xplet_list.npy"]),
                                     allow_pickle=True)
                statistics = get_eff_and_frate(reco_xplets, gen_xplets)
                xi_7_eff_connection_list.append(statistics[0])
                xi_7_frate_connection_list.append(statistics[1])
                
                for chi in statistics[2]:
                    xi_7_eff_connection_list_chi_squared_matched.append(chi)
                for chi in statistics[3]:
                    xi_7_eff_connection_list_chi_squared_fake.append(chi)
                

In [ ]:
plt.figure(figsize=(8, 6), dpi=400)
plt.plot(xi_7_eff_connection_list, label="connection_list", linestyle=" ", marker="o", markersize=4.5)
plt.plot(xi_7_eff_impact_list , label="impact_list", linestyle=" ", marker="s", markersize=4.5)
plt.xlabel("Number of BX", fontsize=16)
plt.ylabel("Efficiency", fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc="best")
plt.title("xi=7, sub-QUBO size=7", fontsize=18, loc="left")
plt.savefig("Connection_list_efficiency.pdf")

In [ ]:
plt.figure(figsize=(8, 6), dpi=400)
plt.plot(xi_7_frate_connection_list, label="connection_list", linestyle=" ", marker="o", markersize=4.5)
plt.plot(xi_7_frate_impact_list , label="impact_list", linestyle=" ", marker="s", markersize=4.5)
plt.xlabel("Number of BX", fontsize=16)
plt.ylabel("Fake rate", fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc="best")
plt.title("xi=7, sub-QUBO size=7", fontsize=18, loc="left")
plt.savefig("Connection_list_fake_rate.pdf")

In [ ]:
plt.hist(xi_7_impact_list_chi_squared_matched)

In [ ]:
print(xi_7_impact_list_chi_squared_matched)